In [1]:
# This Notebook is used to create .pyx files, where we store the functions that we load for ou study
# Many functions come from https://github.com/sdpython/td3a_cpp
# we thank Mr Dupré for these functions

%load_ext Cython

In [2]:
%%cython 
import pyximport
pyximport.install(inplace=True, build_in_temp = False , language_level = 3)

## Sequential module 

In [3]:
%%cython_pyximport lorentz_seqc


#lorentz resolution in cython , is it faster? 
cimport cython

import numpy as np 


cpdef lorentz_resolution_seqc( R, sigma, b, x_0, y_0, z_0, N, T_c, step):
    '''
    Inputs :  R, sigma, b the paramaters of the lorentz system 
              x0, y0, z0 the initial conditions
              N order of the taylor series development
              step the step size between two stages
    Outputs : vectors x,y,z containing all the trajectory of the particle
    
    '''
    # initialisation :  create empty array
    n_steps = int(T_c / step )+1
    x = np.zeros(n_steps + 1 )
    y = np.zeros(n_steps + 1)
    z = np.zeros(n_steps + 1)
    
    x[0] = x_0
    y[0] = y_0
    z[0] = z_0
    
    tau = np.ascontiguousarray( step**np.arange(1,N+1), dtype = np.float64 )

    
    for t in range(1, (n_steps +1 )) : #loop over time
        
        #initialisation : store alpha1, beta1, gamma1
        alpha = np.zeros(N + 1)
        beta = np.zeros(N + 1)
        gamma = np.zeros(N + 1)
        
        alpha[0] = x[t-1]
        beta[0] = y[t-1]
        gamma[0] = z[t-1]
        
        alpha[1] = sigma * (-x[t-1] + y[t-1])
        beta[1] = R * x[t-1] - y[t-1] -x[t-1] * z[t-1]
        gamma[1] = x[t-1] * y[t-1] - b * z[t-1]
        
        for i in range(1, N ) : #loop over derivatives, [::-1] to invert the order of the elements of the vector
            
            div = 1/(i+1)
            alpha[i+1] = div * sigma * ( beta[i]  - alpha[i] )  
            beta[i+1] = div * ( R * alpha[i] - beta[i] - np.sum(alpha[:(i+1)][::-1] * gamma[:(i+1)]  ))
            gamma[i+1] = div * ( np.sum( alpha[:(i+1)][::-1] * beta[ :(i+1)] ) - b * gamma[i] ) 
        
        x[t] = x[t-1] + np.sum( alpha[1:] * tau )
        y[t] = y[t-1] + np.sum( beta[1:] * tau )
        z[t] = z[t-1] + np.sum( gamma[1:] * tau )
        
    return(x,y,z)




## Parallized code :  as in the research paper 

In [3]:
%%cython_pyximport lorentz_omp

cimport cython
import numpy as np


cimport openmp
from cython.parallel cimport prange
from cython.parallel cimport parallel


@cython.boundscheck(False)

@cython.wraparound(False)

cpdef parallel_2ddot(const double[::1] A, const double[::1] B, 
                const double[::1] C, const double[::1] D , 
                 int chunksize, int schedule ):
    
    cdef int n = A.shape[0]
    cdef int i
    cdef double R[2] 
    if schedule == 1:
        with nogil:
            for i in prange(n, schedule='static', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
    elif schedule == 2:
        with nogil :
            for i in prange(n, schedule='dynamic', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
    else:
        with nogil :
            for i in prange(n):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
                
    return R





cpdef lorentz_resolution_omp( R, sigma, b, x_0, y_0, z_0, N, T_c, step, cython.int chunksize=32, cython.int schedule=0):
    '''
    Inputs :  R, sigma, b the paramaters of the lorentz system 
              x0, y0, z0 the initial conditions
              N order of the taylor series development
              step the step size between two stages
    Outputs : vectors x,y,z containing all the trajectory of the particle
    
    '''
    # initialisation :  create empty array
    n_steps = int(T_c / step )+1
    x = np.zeros(n_steps + 1 )
    y = np.zeros(n_steps + 1)
    z = np.zeros(n_steps + 1)
    
    x[0] = x_0
    y[0] = y_0
    z[0] = z_0
    
    tau = np.ascontiguousarray( step**np.arange(1,N+1), dtype = np.float64 )


    
    for t in range(1, (n_steps +1 )) : #loop over time
        
        #initialisation : store alpha1, beta1, gamma1
        
        alpha = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        beta = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        gamma = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        
        alpha[0] = x[t-1]
        beta[0] = y[t-1]
        gamma[0] = z[t-1]
        
        alpha[1] = sigma * (-x[t-1] + y[t-1])
        beta[1] = R * x[t-1] - y[t-1] -x[t-1] * z[t-1]
        gamma[1] = x[t-1] * y[t-1] - b * z[t-1]
        
        for i in range(1, N ) : #loop over derivatives, [::-1] to invert the order of the elements of the vector
            
            
            s =  parallel_2ddot( np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64) 
                                    , np.ascontiguousarray(gamma[:(i+1)], dtype = np.float64)
                                    , np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64)
                                    , np.ascontiguousarray(beta[ :(i+1)] , dtype=np.float64)
                                    , chunksize, schedule) 
            
            div = 1/(i+1)
            alpha[i+1] = div * sigma * ( beta[i]  - alpha[i] )  
            beta[i+1] = div * ( R * alpha[i] - beta[i] - s[0] )     
            gamma[i+1] = div * ( s[1] - b * gamma[i]  )
        
        x[t] = x[t-1] + np.sum( alpha[1:] * tau )
        y[t] = y[t-1] + np.sum( beta[1:] * tau )
        z[t] = z[t-1] + np.sum( gamma[1:] * tau )
        
    return(x,y,z)

## Fully Parallelized code : 3 last dot products parallelized : negligeable against $N^2$

In [4]:
%%cython_pyximport lorentz_ompfull

# we each ddot product is parallelized

cimport cython
import numpy as np
cimport openmp
from cython.parallel cimport prange
from cython.parallel cimport parallel


@cython.boundscheck(False)

@cython.wraparound(False)

cpdef parallel_2ddot(const double[::1] A, const double[::1] B, 
                const double[::1] C, const double[::1] D , 
                 int chunksize, int schedule ):
    
    cdef int n = A.shape[0]
    cdef int i
    cdef double R[2] 
    if schedule == 1:
        with nogil:
            for i in prange(n, schedule='static', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
    elif schedule == 2:
        with nogil :
            for i in prange(n, schedule='dynamic', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
    else:
        with nogil :
            for i in prange(n):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
                
    return R


@cython.boundscheck(False)

@cython.wraparound(False)

cpdef parallel_3ddot(const double[::1] A, const double[::1] B, 
                const double[::1] C, const double[::1] D , 
             const double[::1] E, const double[::1] F , 
                 int chunksize, int schedule ):
    
    cdef int n = A.shape[0]
    cdef int i
    cdef double R[3] 
    if schedule == 1:
        with nogil:
            for i in prange(n, schedule='static', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
                R[2] += E[i] * F[i]
    elif schedule == 2:
        with nogil :
            for i in prange(n, schedule='dynamic', chunksize=chunksize):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
                R[2] += E[i] * F[i]
    else:
        with nogil :
            for i in prange(n):
                R[0] += A[i] * B[i]
                R[1] += C[i] * D[i]
                R[2] += E[i] * F[i]
                
    return R


@cython.boundscheck(False)

@cython.wraparound(False)

cpdef lorentz_resolution_ompfull( R, sigma, b, x_0, y_0, z_0, N, T_c, step, cython.int chunksize=32, cython.int schedule=0):
    '''
    Inputs :  R, sigma, b the paramaters of the lorentz system 
              x0, y0, z0 the initial conditions
              N order of the taylor series development
              step the step size between two stages
    Outputs : vectors x,y,z containing all the trajectory of the particle
    
    '''
    # initialisation :  create empty array
    n_steps = int(T_c / step )+1
    x = np.zeros(n_steps + 1 )
    y = np.zeros(n_steps + 1)
    z = np.zeros(n_steps + 1)
    
    x[0] = x_0
    y[0] = y_0
    z[0] = z_0
    
    tau = np.ascontiguousarray( step**np.arange(1,N+1), dtype = np.float64 )


    
    for t in range(1, (n_steps +1 )) : #loop over time
        
        #initialisation : store alpha1, beta1, gamma1
        
        alpha = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        beta = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        gamma = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        
        alpha[0] = x[t-1]
        beta[0] = y[t-1]
        gamma[0] = z[t-1]
        
        alpha[1] = sigma * (-x[t-1] + y[t-1])
        beta[1] = R * x[t-1] - y[t-1] -x[t-1] * z[t-1]
        gamma[1] = x[t-1] * y[t-1] - b * z[t-1]
        
        for i in range(1, N ) : #loop over derivatives, [::-1] to invert the order of the elements of the vector
            
            
            s =  parallel_2ddot( np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64) 
                                    , np.ascontiguousarray(gamma[:(i+1)], dtype = np.float64)
                                    , np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64)
                                    , np.ascontiguousarray(beta[ :(i+1)] , dtype=np.float64)
                                    , chunksize, schedule) 
            
            div = 1/(i+1)
            alpha[i+1] = div * sigma * ( beta[i]  - alpha[i] )  
            beta[i+1] = div * ( R * alpha[i] - beta[i] - s[0] )
            
            gamma[i+1] = div * ( s[1] - b * gamma[i]  )
        

# parallelized end of the code 

        r = parallel_3ddot(np.ascontiguousarray(alpha[1:], dtype=np.float64) , tau
                                    , np.ascontiguousarray(beta[1:], dtype=np.float64) , tau
                                    , np.ascontiguousarray(gamma[1:], dtype=np.float64) , tau
                                    , chunksize, schedule) 
        
        x[t] = x[t-1] + r[0]
        y[t] = y[t-1] + r[1]
        z[t] = z[t-1] + r[2]
        
    return(x,y,z)



## Parallelized sequentially  

In [ ]:
%%cython_pyximport lorentz_seqp


### We wanted to see the results of sequentially parallelized dot products. 
## This configuration is contrary to the principle of parallelization. 
## results are not shown in the notebook

cimport cython
from cython.parallel import prange, parallel
import numpy as np

@cython.boundscheck(False)

cdef double _ddot_cython_array_omp(const double[::1] va, const double[::1] vb,
                                   int chunksize, int schedule) nogil:
    """
    dot product implemented with cython and C types
    using :epkg:`prange` (:epkg:`openmp` in :epkg:`cython`).
    
    :param va: first vector, dtype must be float64
    :param vb: second vector, dtype must be float64
    :return: dot product
    """
    cdef int n = va.shape[0]
    cdef Py_ssize_t i

    cdef double s = 0
    if schedule == 1:
        for i in prange(n, schedule='static', chunksize=chunksize):
            s += va[i] * vb[i]
    elif schedule == 2:
        for i in prange(n, schedule='dynamic', chunksize=chunksize):
            s += va[i] * vb[i]
    else:
        for i in prange(n):
            s += va[i] * vb[i]
    return s


def ddot_cython_array_omp(const double[::1] va, const double[::1] vb,
                          cython.int chunksize=32, cython.int schedule=0):
    """
    dot product implemented with cython and C types
    using :epkg:`prange` (:epkg:`openmp` in :epkg:`cython`).
    
    :param va: first vector, dtype must be float64
    :param vb: second vector, dtype must be float64
    :param chunksize: see :epkg:`prange`
    :param schedule: see :epkg:`prange`
    :return: dot product
    """
    if va.shape[0] != vb.shape[0]:        
        raise ValueError("Vectors must have same shape.")
    cdef double s
    with nogil:
        s = _ddot_cython_array_omp(va, vb, chunksize, schedule)
    return s




def lorentz_resolution_seqp( R, sigma, b, x_0, y_0, z_0, N, T_c, step, cython.int chunksize=32, cython.int schedule=0):
    '''
    Inputs :  R, sigma, b the paramaters of the lorentz system 
              x0, y0, z0 the initial conditions
              N order of the taylor series development
              step the step size between two stages
    Outputs : vectors x,y,z containing all the trajectory of the particle
    
    '''
    # initialisation :  create empty array
    n_steps = int(T_c / step )+1
    x = np.zeros(n_steps + 1 )
    y = np.zeros(n_steps + 1)
    z = np.zeros(n_steps + 1)
    
    x[0] = x_0
    y[0] = y_0
    z[0] = z_0
    
    tho = np.ascontiguousarray( step**np.arange(1,N+1), dtype = np.float64 )


    
    for t in range(1, (n_steps +1 )) : #loop over time
        
        #initialisation : store alpha1, beta1, gamma1
        
        alpha = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        beta = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        gamma = np.ascontiguousarray(np.zeros(N + 1) , dtype=np.float64)
        
        alpha[0] = x[t-1]
        beta[0] = y[t-1]
        gamma[0] = z[t-1]
        
        alpha[1] = sigma * (-x[t-1] + y[t-1])
        beta[1] = R * x[t-1] - y[t-1] -x[t-1] * z[t-1]
        gamma[1] = x[t-1] * y[t-1] - b * z[t-1]
        
        for i in range(1, N ) : #loop over derivatives, [::-1] to invert the order of the elements of the vector
            
            div = 1/(i+1)
            alpha[i+1] = div * sigma * ( beta[i]  - alpha[i] )  
            beta[i+1] = div * ( R * alpha[i] - beta[i] - _ddot_cython_array_omp( np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64) 
                                                                                , np.ascontiguousarray(gamma[:(i+1)], dtype = np.float64)
                                                                                , chunksize, schedule) )
            
            gamma[i+1] = div * ( _ddot_cython_array_omp( np.ascontiguousarray(alpha[:(i+1)][::-1], dtype=np.float64) 
                                                        , np.ascontiguousarray(beta[ :(i+1)] , dtype=np.float64)
                                                        , chunksize, schedule )
                                - b * gamma[i]  )
            
        
        x[t] = x[t-1] + _ddot_cython_array_omp( np.ascontiguousarray( alpha[1:] , dtype=np.float64) 
                                                , tho
                                               , chunksize, schedule) 
        y[t] = y[t-1] + _ddot_cython_array_omp( np.ascontiguousarray( beta[1:] , dtype=np.float64) 
                                                , tho
                                               , chunksize, schedule) 
        z[t] = z[t-1] + _ddot_cython_array_omp( np.ascontiguousarray( gamma[1:] , dtype=np.float64) 
                                                , tho
                                               , chunksize, schedule)
        
    return(x,y,z)

